In [6]:
import requests
import pandas as pd
import dash

In [12]:
# Define the URL
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

# Fetch the CSV file from the URL
response = requests.get(URL)

# Read the CSV file into a DataFrame
spacex_df = pd.read_csv(URL)

# Rename the column from "Payload Mass (kg)" to "PayloadMass"
spacex_df.rename(columns={"Payload Mass (kg)": "PayloadMass"}, inplace=True)

# Display the DataFrame
spacex_df.head()


,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,PayloadMass,Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [6]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import pandas as pd
import requests
import io

# Fetch the CSV file from the URL
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
response = requests.get(URL)
spacex_df = pd.read_csv(io.StringIO(response.text))

# Rename the column from "Payload Mass (kg)" to "PayloadMass"
spacex_df.rename(columns={"Payload Mass (kg)": "PayloadMass"}, inplace=True)

# Consolidate Booster Version to just model and series
def consolidate_booster_version(version):
    parts = version.split(' ')
    return ' '.join(parts[:2])

spacex_df['Booster Version'] = spacex_df['Booster Version'].apply(consolidate_booster_version)

# Initialize the Dash app
app = Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Sites', 'value': 'ALL'},
            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
        ],
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    dcc.Graph(id='success-pie-chart'),
    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=1000,
        marks={i: f'{i}' for i in range(0, 10001, 1000)},
        value=[0, 10000]
    ),
    dcc.Graph(id='payload-scatter-chart')
])

# Callback function to update the pie chart
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', 
                     names='Launch Site', 
                     title='Total Success Launches By Site')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig = px.pie(filtered_df, names='class', 
                     title=f'Total Success Launches for site {entered_site}')
        return fig

# Callback function to update the scatter plot based on payload range and launch site
@app.callback(
    Output(component_id='payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def update_scatter_chart(entered_site, payload_range):
    low, high = payload_range
    filtered_df = spacex_df[(spacex_df['PayloadMass'] >= low) & (spacex_df['PayloadMass'] <= high)]

    if entered_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]

    # Map class values to labels
    filtered_df['class'] = filtered_df['class'].map({1: 'Success', 0: 'Failure'})

    fig = px.scatter(filtered_df, x='PayloadMass', y='class', color='Booster Version',
                     title='Correlation between Payload and Success for selected site')
    return fig

# Run the app on port 8051
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)